<a href="https://colab.research.google.com/github/DawenZhang/online_review_intelligent_kano/blob/master/product_review_kano.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#@markdown # product feature selection

from collections import OrderedDict
import numpy as np

product_feature_1 = "" #@param {type:"string"}
product_feature_1_color = "df6953" #@param {type:"string"}

product_feature_2 = "" #@param {type:"string"}
product_feature_2_color = "a1bc35" #@param {type:"string"}

product_feature_3 = "" #@param {type:"string"}
product_feature_3_color = "22afbe" #@param {type:"string"}

product_feature_4 = "" #@param {type:"string"}
product_feature_4_color = "a3a0ec" #@param {type:"string"}

product_feature_5 = "" #@param {type:"string"}
product_feature_5_color = "e4b858" #@param {type:"string"}

product_feature_6 = "" #@param {type:"string"}
product_feature_6_color = "469c6d" #@param {type:"string"}

product_feature_7 = "" #@param {type:"string"}
product_feature_7_color = "8b674e" #@param {type:"string"}

product_feature_8 = "" #@param {type:"string"}
product_feature_8_color = "d772c0" #@param {type:"string"}

product_feature_9 = "" #@param {type:"string"}
product_feature_9_color = "f1b8a4" #@param {type:"string"}

product_feature_10 = "" #@param {type:"string"}
product_feature_10_color = "005e9f" #@param {type:"string"}

product_features = OrderedDict({
    product_feature_1: product_feature_1.split("/"),
    product_feature_2: product_feature_2.split("/"),
    product_feature_3: product_feature_3.split("/"),
    product_feature_4: product_feature_4.split("/"),
    product_feature_5: product_feature_5.split("/"),
    product_feature_6: product_feature_6.split("/"),
    product_feature_7: product_feature_7.split("/"),
    product_feature_8: product_feature_8.split("/"),
    product_feature_9: product_feature_9.split("/"),
    product_feature_10: product_feature_10.split("/")
})

color_array = np.array([
    "#" + product_feature_1_color,
    "#" + product_feature_2_color,
    "#" + product_feature_3_color,
    "#" + product_feature_4_color,
    "#" + product_feature_5_color,
    "#" + product_feature_6_color,
    "#" + product_feature_7_color,
    "#" + product_feature_8_color,
    "#" + product_feature_9_color,
    "#" + product_feature_10_color
         ])

In [0]:
#@markdown # data preparation

feature_vocabulary = []
for feature_name in product_features:
    for meta_feature_name in product_features[feature_name]:
        feature_vocabulary.append(meta_feature_name)
       
      
def check_features(something, product_features):
    feature_list = []
    for i in product_features:
        for feature_tag in product_features[i]:
            if something.lower() == feature_tag:
                feature_list.append(i)
    return feature_list
  
  
#@markdown the product used for anomaly detection
product_id = 1 #@param {type:"integer"}

import pandas as pd

#@markdown the datasheet path; if you wish to upload files, leave them blank
customer_reviews_datasheet_path = "" #@param {type:"string"}
entity_sentiment_datasheet_path = "" #@param {type:"string"}

if customer_reviews_datasheet_path == "":
  from google.colab import files
  uploaded = {}
  while len([*uploaded.keys()]) <= 0:
    print("as the path field is left blank, please upload customer_reviews_datasheet")
    uploaded = files.upload()
  import io
  customer_reviews = pd.read_csv(io.StringIO(uploaded[[*uploaded.keys()][0]].decode('utf-8')))
else:
  customer_reviews = pd.read_csv(customer_reviews_datasheet_path)

  customer_reviews['review_content'] = customer_reviews['review_content'].astype(str)
product_reviews = customer_reviews.loc[customer_reviews['product_id'] == product_id, ['review_id', 'review_content']].sort_values(by = ['review_id'])

replaced_product_reviews = []
for r_index, review in product_reviews.iterrows():
    replaced_review = review[1]
    for pf_name in product_features:
      if len(product_features[pf_name]) > 1:
        for pf_e_name in product_features[pf_name][1:]:
          replaced_review = replaced_review.replace(pf_e_name, product_features[pf_name][0])
    replaced_product_reviews.append(replaced_review)

from sklearn.feature_extraction.text import TfidfVectorizer

tf_features = []

for feature_name in product_features:
    tf_features.append(product_features[feature_name][0])
    
vectorizer = TfidfVectorizer(vocabulary = tf_features, lowercase=True, ngram_range=(1, 2))
tfidf = vectorizer.fit_transform(replaced_product_reviews)

feature_entities = {}


if entity_sentiment_datasheet_path == "":
  from google.colab import files
  uploaded = {}
  while len([*uploaded.keys()]) <= 0:
    print("as the path field is left blank, please upload entity_sentiment_datasheet")
    uploaded = files.upload()
  import io
  entity_sentiment = pd.read_csv(io.StringIO(uploaded[[*uploaded.keys()][0]].decode('utf-8')))
else:
  entity_sentiment = pd.read_csv(entity_sentiment_datasheet_path)

for review_num in range(len(replaced_product_reviews)):
    current_all_entities = entity_sentiment.loc[
                                                (entity_sentiment['review_id'] == product_reviews.iloc[review_num]['review_id']) 
                                                & (entity_sentiment['product_id'] == product_id), 
                                                ['name', 'sentiment_score', 'sentiment_magnitude']
                                                ]
    current_entities = {}
    
    for e_index, entity in current_all_entities.iterrows():
        
        features = check_features(entity['name'], product_features)

        if len(features) > 0:

            if entity['sentiment_magnitude'] == 0:
                continue
            
            pending_entity = []
            
            pending_entity.append(tfidf[review_num, [*product_features].index(features[0])])
            pending_entity.append(entity[1])
            pending_entity.append(entity[2])
            
            for single_feature in features:
                if single_feature not in current_entities:
                    current_entities[single_feature] = []
                current_entities[single_feature].append(pending_entity)
                
    for feature_name in current_entities:
        if feature_name not in feature_entities:
            feature_entities[feature_name] = []
        adding_entity = [0, 0, 0]
        for meta_feature_entity in current_entities[feature_name]:
            adding_entity[0] += meta_feature_entity[0]
            adding_entity[1] += meta_feature_entity[1]
            adding_entity[2] += meta_feature_entity[2]
            
        adding_entity[0] = adding_entity[0] / len(current_entities[feature_name])
        adding_entity[1] = adding_entity[1] / len(current_entities[feature_name])

        feature_entities[feature_name].append(adding_entity)
        
from scipy import stats

features_average = []
for feature_name in feature_entities:
    positive_sum = 0
    positive_count = 0
    negative_sum = 0
    negative_count = 0
    magnitude_sum = 0
    magnitude_count = 0
    
    for feature_entity in feature_entities[feature_name]: 
        if feature_entity[1] > 0:
            positive_sum += (feature_entity[1]  * feature_entity[0])
            positive_count += 1
        elif feature_entity[1] < 0:
            negative_sum += (-feature_entity[1] * feature_entity[0])
            negative_count += 1
        magnitude_sum += feature_entity[2] * feature_entity[0]
        magnitude_count += 1

    features_average.append([
                             0 if positive_count == 0 else positive_sum/positive_count,
                             0 if negative_count == 0 else negative_sum/negative_count,
                             0 if magnitude_count == 0 else magnitude_sum/magnitude_count,
                             feature_name
                            ])
    
features_out_of_order = {}
for feature_out_of_order in features_average:
    features_out_of_order[feature_out_of_order[3]] = feature_out_of_order
    
features_in_order = []
for feature in product_features:
    if feature not in features_out_of_order:
      features_in_order.append([0, 0, 0, feature])
    else:
      features_in_order.append(features_out_of_order[feature])

kano_array = np.array(features_in_order)

In [0]:
#@markdown # plotting

import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.add_subplot(111)
ax.axis('off')

colortable_title = "Color Table" #@param {type:"string"}
ax.set_title(colortable_title)
ax.table(cellText=[
                    [feature_x for feature_x in [*product_features][0:5]], ['' for color_text_y in range(0, 5)],
                    [feature_x for feature_x in [*product_features][5:10]], ['' for color_text_y in range(0, 5)]
                   ], 
          cellLoc = 'center', loc = 'center',
          colWidths = [0.25, 0.25, 0.25, 0.25, 0.25], cellColours = [['w' for color_y in range(0, 5)], color_array[0:5], ['w' for color_y in range(0, 5)], color_array[5:10]]).scale(1, 1.2)

plt.subplots_adjust(left = 0.2, top = 0.8)
plt.show()

#@title select angles and radius for KANO
positive_angle = 23.1 #@param {type:"slider", min:0, max:45, step:0.1}
positive_angle_degree = 90 - positive_angle
negative_angle = 28.4 #@param {type:"slider", min:0, max:45, step:0.1}
negative_angle_degree = negative_angle
corner_radius = 0.3 #@param {type:"slider", min:0, max:1, step:0.1}
x_y_axis_range = 1 #@param {type:"slider", min:0, max:1, step:0.1}
z_axis_range = 1 #@param {type:"slider", min:0, max:1, step:0.1}

view_angle_z = 22 #@param {type:"slider", min:0, max:180, step:1}
view_angle_xy = 250 #@param {type:"slider", min:0, max:360, step:1}

fig = plt.figure(figsize = (6, 6))
ax = fig.add_subplot(111)

x_2d = kano_array[:, 1].astype(float)
y_2d = kano_array[:, 0].astype(float)
z_2d = 50

ax.scatter(x_2d, y_2d, z_2d, color = color_array)

kano_2d_title = "2D Kano Classification" #@param {type:"string"}
kano_2d_satisfaction_axis_label = "Customer Satisfaction" #@param {type:"string"}
kano_2d_dissatisfaction_axis_label = "Customer Dissatisfaction" #@param {type:"string"}

ax.set_title(kano_2d_title)
ax.set_xlabel(kano_2d_dissatisfaction_axis_label)
ax.set_ylabel(kano_2d_satisfaction_axis_label)

ax.set_xlim(0, 1)
ax.set_ylim(0, 1)

theta = np.linspace(0, 0.5 * np.pi, 100)
tz = np.linspace(0, 0, 100)
tr = tz ** 2 + corner_radius
tx = tr * np.sin(theta)
ty = tr * np.cos(theta)
ax.plot(tx, ty, color = '#505050', ls='dashed')

bz = np.linspace(0, 0, 100)
br = np.linspace(corner_radius * np.cos(np.radians(negative_angle_degree)), 1, 100)
bx = br
by = br * np.tan(np.radians(negative_angle_degree))
ax.plot(bx, by, color = '#505050', ls = 'dashed')

cz = np.linspace(0, 0, 100)
cr = np.linspace(corner_radius * np.cos(np.radians(positive_angle_degree)), 1 / np.tan(np.radians(positive_angle_degree)), 100)
cx = cr
cy = cr * np.tan(np.radians(positive_angle_degree))
qqq = ax.plot(cx, cy, color = '#505050', ls = 'dashed')
plt.grid(True)

plt.show()

from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(16, 12))
ax = fig.gca(projection='3d')

kano_3d_title = "3D Kano Classification" #@param {type:"string"}
kano_3d_satisfaction_axis_label = "Customer Satisfaction" #@param {type:"string"}
kano_3d_dissatisfaction_axis_label = "Customer Dissatisfaction" #@param {type:"string"}
kano_3d_magnitude_axis_label = "Sentiment Magnitude" #@param {type:"string"}

ax.set_title(kano_3d_title)
ax.set_xlabel(kano_3d_dissatisfaction_axis_label)
ax.set_ylabel(kano_3d_satisfaction_axis_label)
ax.set_zlabel(kano_3d_magnitude_axis_label)

ax.set_xlim(0, x_y_axis_range)
ax.set_ylim(0, x_y_axis_range)
ax.set_zlim(0, z_axis_range)

x_3d = kano_array[:, 1].astype(float)
y_3d = kano_array[:, 0].astype(float)
z_3d = kano_array[:, 2].astype(float)

dx = 0.01 * np.ones_like(x_3d)
dy = 0.01 * np.ones_like(y_3d)
dz = abs(z_3d) * z_3d.flatten()
dz = dz.flatten() / abs(z_3d)
z_3d = np.zeros_like(z_3d)


ax.bar3d(x_3d, y_3d, z_3d - z_axis_range * 0.02, dx * 2, dy * 2, dz, color = color_array, shade = False)

theta = np.linspace(0, 0.5 * np.pi, 100)
tz = np.linspace(0, 0, 100)
tr = tz ** 2 + corner_radius
tx = tr * np.sin(theta)
ty = tr * np.cos(theta)
ax.plot(tx, ty, tz - z_axis_range * 0.02, color = '#505050', ls = 'dashed')

bz = np.linspace(0, 0, 100)
br = np.linspace(corner_radius * np.cos(np.radians(negative_angle_degree)), x_y_axis_range, 100)
bx = br
by = br * np.tan(np.radians(negative_angle_degree))
ax.plot(bx, by, bz - z_axis_range * 0.02, color = '#505050', ls = 'dashed')

cz = np.linspace(0, 0, 100)
cr = np.linspace(corner_radius * np.cos(np.radians(positive_angle_degree)), x_y_axis_range / np.tan(np.radians(positive_angle_degree)), 100)
cx = cr
cy = cr * np.tan(np.radians(positive_angle_degree))
ax.plot(cx, cy, cz - z_axis_range * 0.02, color = '#505050', ls = 'dashed')

ax.view_init(view_angle_z, view_angle_xy)

plt.show()